# NB06.5: Universal Kinematic Cleaning Lab

**Purpose**: A general-purpose cleaning gate that applies to any session without hard-coding specific joint names.

**Pipeline Stages**:
1. **Adaptive Artifact Detection** - Find physically impossible values automatically
2. **Multi-Path Repair** - PCHIP for scalars, SLERP for quaternions
3. **Recalculation & Consistency** - Rebuild acceleration from cleaned velocity
4. **Universal Comparison Log** - Before/After audit table

**Input**: Output from NB06 (`df_combined_kinematics.parquet`)

**Output**: Cleaned kinematics ready for NB07 audit

---
**Physical Ceilings** (Literature-based):
- Angular Velocity: 1200 °/s (peak human joint velocity)
- Angular Acceleration: 50,000 °/s² (biomechanical limit)
- ROM: 250° (physiological joint limit for most human joints)

In [ ]:
# --- CELL 01: Setup & Configuration ---
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from scipy.interpolate import PchipInterpolator
from scipy.spatial.transform import Rotation as R, Slerp

# --- Path Setup (Consistent with other notebooks) ---
if os.path.basename(os.getcwd()) == 'notebooks':
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
else:
    PROJECT_ROOT = os.path.abspath(os.getcwd())
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

from pipeline_config import CONFIG

# Derive filename from config (synchronized with previous notebooks)
csv_filename = Path(CONFIG['current_csv']).stem
RUN_ID = csv_filename
TARGET_FPS = CONFIG['FS_TARGET']

# Paths (consistent with pipeline structure)
DERIVATIVES = Path(PROJECT_ROOT) / CONFIG['derivatives_dir']
STEP_06_DIR = DERIVATIVES / 'step_06_kinematics'
STEP_065_DIR = DERIVATIVES / 'step_065_cleaned'
STEP_065_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================
# PHYSICAL CEILINGS (Universal - No Hard-Coded Joint Names)
# ============================================================
CEILING_ANGULAR_VEL = 1200.0      # deg/s - Peak human joint velocity
CEILING_ANGULAR_ACC = 50000.0     # deg/s² - Biomechanical acceleration limit
CEILING_ROM = 250.0               # degrees - Max physiological joint ROM

print(f"{'='*70}")
print(f"🔬 NB06.5: Universal Kinematic Cleaning Lab")
print(f"{'='*70}")
print(f"Session: {RUN_ID}")
print(f"Sample Rate: {TARGET_FPS} Hz")
print(f"\n📏 Physical Ceilings:")
print(f"   Angular Velocity:     {CEILING_ANGULAR_VEL:,.0f} °/s")
print(f"   Angular Acceleration: {CEILING_ANGULAR_ACC:,.0f} °/s²")
print(f"   ROM Limit:            {CEILING_ROM:.0f}°")
print(f"{'='*70}")

In [ ]:
# --- CELL 02: Load Data from NB06 ---
input_path = STEP_06_DIR / f"{RUN_ID}__kinematics.parquet"

if not input_path.exists():
    raise FileNotFoundError(f"Input file not found: {input_path}\nPlease run NB06 first.")

df_raw = pd.read_parquet(input_path)
df = df_raw.copy()  # Working copy for cleaning

print(f"✅ Loaded: {input_path.name}")
print(f"   Shape: {df.shape[0]:,} frames × {df.shape[1]:,} columns")
print(f"   Duration: {df.shape[0] / TARGET_FPS:.2f} seconds")

# ============================================================
# Auto-Detect Column Types (No Hard-Coded Names)
# ============================================================
vel_cols = [c for c in df.columns if c.endswith('_vel') or '_vel' in c]
acc_cols = [c for c in df.columns if c.endswith('_acc') or '_acc' in c]
mag_vel_cols = [c for c in df.columns if 'mag_vel' in c]
quat_cols = [c for c in df.columns if any(c.endswith(s) for s in ['__qx', '__qy', '__qz', '__qw'])]
# IMPORTANT: Angle columns for ROM = only position angles, NOT velocity/acceleration
angle_cols = [c for c in df.columns if '_angle' in c.lower() and '_vel' not in c.lower() and '_acc' not in c.lower()]

# Extract unique joint names from velocity columns
joint_names = list(set([c.rsplit('_', 2)[0] for c in mag_vel_cols if 'mag_vel' in c]))

print(f"\n📊 Auto-Detected Columns:")
print(f"   Velocity columns:     {len(vel_cols)}")
print(f"   Acceleration columns: {len(acc_cols)}")
print(f"   Magnitude velocity:   {len(mag_vel_cols)}")
print(f"   Quaternion columns:   {len(quat_cols)}")
print(f"   Angle columns:        {len(angle_cols)}")
print(f"   Unique joints:        {len(joint_names)}")

In [ ]:
# --- CELL 02.5: Angle Unwrapping (Fix ±180° Phase Jumps) ---
# Unwrap angles to ensure ROM reflects human envelope, not math envelope
# Fixes cases where 179° → -179° appears as 358° jump (actually just 2°)

print(f"\n{'='*70}")
print(f"🔄 ANGLE UNWRAPPING: Fixing ±180° Phase Jumps")
print(f"{'='*70}")

def robust_unwrap(series: pd.Series) -> pd.Series:
    """
    Unwrap angle data to remove ±180° phase jumps.
    
    Uses np.unwrap to convert degrees to radians, unwrap, then convert back.
    This ensures ROM reflects actual human motion, not mathematical wraparound.
    
    Example:
        Raw: [179°, 180°, -179°] → ROM = 358° (WRONG - phase jump)
        Unwrapped: [179°, 180°, 181°] → ROM = 2° (CORRECT - actual motion)
    """
    # Convert to radians, unwrap, convert back to degrees
    angles_rad = np.deg2rad(series.values)
    unwrapped_rad = np.unwrap(angles_rad)
    unwrapped_deg = np.rad2deg(unwrapped_rad)
    
    return pd.Series(unwrapped_deg, index=series.index)

# Helper function for ROM calculation
def calculate_joint_rom(angles: np.ndarray, use_wrapped: bool = False) -> float:
    """
    Calculate Range of Motion from angle data.
    
    Args:
        angles: Angle values (can be wrapped or unwrapped)
        use_wrapped: If True, wrap angles to [-180, 180] before calculating ROM
                     This prevents cumulative rotation from inflating ROM
    
    Returns:
        ROM in degrees (actual range of motion, not cumulative rotation)
    """
    valid_angles = angles[np.isfinite(angles)]
    if len(valid_angles) < 2:
        return 0.0
    
    # Calculate ROM on unwrapped angles (phase jumps already fixed by unwrapping)
    rom_unwrapped = float(np.max(valid_angles) - np.min(valid_angles))
    
    # If ROM > 360°, it means the joint rotated more than one full cycle
    # In this case, wrap to get the actual range within one cycle
    if rom_unwrapped > 360:
        # Multiple full rotations - wrap to get actual ROM within one cycle
        angles_wrapped = np.mod(valid_angles + 180, 360) - 180
        rom_wrapped = float(np.max(angles_wrapped) - np.min(angles_wrapped))
        # Return the wrapped ROM (actual range within one cycle)
        return rom_wrapped
    elif rom_unwrapped > 350:
        # ROM between 350-360° is suspicious - likely a phase jump artifact
        # Check if there's a discontinuity (large jump in the middle)
        sorted_angles = np.sort(valid_angles)
        diffs = np.diff(sorted_angles)
        max_gap = np.max(diffs)
        
        # If there's a large gap (> 10°), it's likely a phase jump
        # The actual ROM is probably the smaller of the two ranges
        if max_gap > 10:
            # Find the gap and calculate ROM on each side
            gap_idx = np.argmax(diffs)
            range1 = sorted_angles[gap_idx] - sorted_angles[0]
            range2 = sorted_angles[-1] - sorted_angles[gap_idx + 1]
            # Return the larger range (actual motion range)
            actual_rom = max(range1, range2)
            return actual_rom
        
        # No large gap - might be real full rotation, but for joints this is unlikely
        # Wrap to get the range within one cycle
        angles_wrapped = np.mod(valid_angles + 180, 360) - 180
        rom_wrapped = float(np.max(angles_wrapped) - np.min(angles_wrapped))
        return rom_wrapped
    else:
        # ROM < 350° means it's within one rotation cycle - use unwrapped value
        # This correctly handles phase jumps that were fixed by unwrapping
        return rom_unwrapped

# Store original ROMs for comparison
rom_before_unwrap = {}
rom_after_unwrap = {}

if angle_cols:
    print(f"\n📍 Unwrapping {len(angle_cols)} angle columns...")
    
    unwrap_stats = {
        'columns_unwrapped': 0,
        'rom_reductions': [],
        'total_rom_reduction': 0.0
    }
    
    for col in angle_cols:
        # Calculate ROM before unwrapping (use wrapped to get actual ROM)
        angles_raw = df[col].values
        rom_raw = calculate_joint_rom(angles_raw, use_wrapped=True)
        rom_before_unwrap[col] = rom_raw
        
        # Unwrap the angles (for data continuity)
        angles_unwrapped = robust_unwrap(df[col])
        df[col] = angles_unwrapped.values
        
        # Calculate ROM after unwrapping (directly on unwrapped angles)
        # Unwrapping already fixed phase jumps, so calculate ROM directly
        rom_unwrapped = calculate_joint_rom(df[col].values, use_wrapped=False)
        rom_after_unwrap[col] = rom_unwrapped
        
        # Track significant reductions (phase jumps fixed)
        if rom_raw > 250 and rom_unwrapped < rom_raw * 0.8:  # >20% reduction
            reduction = rom_raw - rom_unwrapped
            unwrap_stats['rom_reductions'].append({
                'column': col,
                'raw_rom': rom_raw,
                'unwrapped_rom': rom_unwrapped,
                'reduction': reduction
            })
            unwrap_stats['total_rom_reduction'] += reduction
        
        if rom_unwrapped != rom_raw:
            unwrap_stats['columns_unwrapped'] += 1
    
    print(f"   ✅ Unwrapped {unwrap_stats['columns_unwrapped']} columns with phase jumps")
    
    if unwrap_stats['rom_reductions']:
        print(f"\n   📊 Phase Jumps Fixed (ROM Reduction >20%):")
        print(f"   {'-'*90}")
        print(f"   {'Column':<50} {'Raw ROM':>12} {'Unwrapped ROM':>15} {'Reduction':>12}")
        print(f"   {'-'*90}")
        
        for stat in sorted(unwrap_stats['rom_reductions'], key=lambda x: -x['reduction'])[:10]:
            print(f"   {stat['column']:<50} {stat['raw_rom']:>10.1f}° {stat['unwrapped_rom']:>13.1f}° {stat['reduction']:>10.1f}°")
        
        print(f"   {'-'*90}")
        print(f"   Total ROM reduction: {unwrap_stats['total_rom_reduction']:.1f}°")
        print(f"   ✅ Angles now reflect human envelope, not math envelope")
    else:
        print(f"   ✅ No significant phase jumps detected")
    
    # Debug: Check elbow angles specifically
    elbow_cols = [c for c in angle_cols if 'elbow' in c.lower() and 'angle' in c.lower()]
    if elbow_cols:
        print(f"\n   🔍 DEBUG: Elbow Angle Analysis")
        print(f"   {'-'*90}")
        for col in elbow_cols[:4]:  # Check first 4 elbow columns
            angles = df[col].values
            angles_wrapped = np.mod(angles + 180, 360) - 180
            rom_wrapped = float(np.max(angles_wrapped) - np.min(angles_wrapped))
            rom_unwrapped = float(np.max(angles) - np.min(angles))
            
            # Check for phase jumps (large jumps between consecutive values)
            diffs = np.abs(np.diff(angles))
            phase_jumps = np.sum(diffs > 300)  # Jumps > 300° are likely phase jumps
            
            print(f"   {col}:")
            print(f"      Wrapped ROM: {rom_wrapped:.1f}° | Unwrapped ROM: {rom_unwrapped:.1f}°")
            print(f"      Min: {np.min(angles_wrapped):.1f}° | Max: {np.max(angles_wrapped):.1f}°")
            print(f"      Phase jumps detected: {phase_jumps}")
            if phase_jumps > 0:
                jump_indices = np.where(diffs > 300)[0]
                print(f"      Jump locations: {jump_indices[:5]} (showing first 5)")
        print(f"   {'-'*90}")
else:
    print(f"   ⚠️ No angle columns found - skipping unwrapping")

print(f"\n{'='*70}")

In [ ]:
# --- CELL 03: STAGE 1 - Adaptive Artifact Detection ---
# Automatically find ANY joint with physically impossible values

print(f"\n{'='*70}")
print(f"🔍 STAGE 1: Adaptive Artifact Detection")
print(f"{'='*70}")

# Track artifacts for audit
artifacts_detected = {
    'velocity': {},
    'acceleration': {},
    'total_frames_affected': 0
}

# --- 1A: Velocity Artifacts (>1200 °/s) ---
print(f"\n📍 Scanning for Velocity Artifacts (>{CEILING_ANGULAR_VEL}°/s)...")
velocity_artifact_mask = pd.DataFrame(index=df.index)

for col in vel_cols:
    artifact_mask = np.abs(df[col]) > CEILING_ANGULAR_VEL
    n_artifacts = artifact_mask.sum()
    
    if n_artifacts > 0:
        velocity_artifact_mask[col] = artifact_mask
        max_val = df.loc[artifact_mask, col].abs().max() if artifact_mask.any() else 0
        artifacts_detected['velocity'][col] = {
            'count': int(n_artifacts),
            'max_value': float(max_val),
            'percent': float(n_artifacts / len(df) * 100)
        }
        # Mark as NaN for repair
        df.loc[artifact_mask, col] = np.nan

# --- 1B: Acceleration Artifacts (>50,000 °/s²) ---
print(f"📍 Scanning for Acceleration Artifacts (>{CEILING_ANGULAR_ACC:,.0f}°/s²)...")
acceleration_artifact_mask = pd.DataFrame(index=df.index)

for col in acc_cols:
    artifact_mask = np.abs(df[col]) > CEILING_ANGULAR_ACC
    n_artifacts = artifact_mask.sum()
    
    if n_artifacts > 0:
        acceleration_artifact_mask[col] = artifact_mask
        max_val = df.loc[artifact_mask, col].abs().max() if artifact_mask.any() else 0
        artifacts_detected['acceleration'][col] = {
            'count': int(n_artifacts),
            'max_value': float(max_val),
            'percent': float(n_artifacts / len(df) * 100)
        }
        # Mark as NaN for repair
        df.loc[artifact_mask, col] = np.nan

# Summary
total_vel_artifacts = sum(v['count'] for v in artifacts_detected['velocity'].values())
total_acc_artifacts = sum(v['count'] for v in artifacts_detected['acceleration'].values())
artifacts_detected['total_frames_affected'] = total_vel_artifacts + total_acc_artifacts

print(f"\n📊 STAGE 1 Results:")
print(f"   Velocity artifacts:     {total_vel_artifacts:,} frames in {len(artifacts_detected['velocity'])} columns")
print(f"   Acceleration artifacts: {total_acc_artifacts:,} frames in {len(artifacts_detected['acceleration'])} columns")

if artifacts_detected['velocity']:
    print(f"\n   🚨 Joints with Velocity Artifacts:")
    for col, info in sorted(artifacts_detected['velocity'].items(), key=lambda x: -x[1]['count'])[:10]:
        print(f"      {col}: {info['count']:,} frames (max: {info['max_value']:.1f}°/s)")

if artifacts_detected['acceleration']:
    print(f"\n   🚨 Joints with Acceleration Artifacts:")
    for col, info in sorted(artifacts_detected['acceleration'].items(), key=lambda x: -x[1]['count'])[:10]:
        print(f"      {col}: {info['count']:,} frames (max: {info['max_value']:.1f}°/s²)")

In [ ]:
# --- CELL 04: STAGE 2A - PCHIP Repair for Scalar Kinematics ---
# Piecewise Cubic Hermite Interpolating Polynomial
# Monotonic cubic spline - no artificial ringing or overshoots

print(f"\n{'='*70}")
print(f"🔧 STAGE 2A: PCHIP Interpolation for Scalar Kinematics")
print(f"{'='*70}")

def pchip_repair(series: pd.Series) -> Tuple[pd.Series, int]:
    """
    Repair NaN gaps using PCHIP interpolation.
    Returns repaired series and count of repaired values.
    """
    nan_mask = series.isna()
    n_nans = nan_mask.sum()
    
    if n_nans == 0:
        return series, 0
    
    if n_nans == len(series):
        # All NaN - cannot interpolate
        return series, 0
    
    # Get valid data points
    valid_idx = np.where(~nan_mask)[0]
    valid_values = series.iloc[valid_idx].values
    
    if len(valid_idx) < 2:
        # Not enough points for interpolation
        return series, 0
    
    try:
        # Create PCHIP interpolator
        pchip = PchipInterpolator(valid_idx, valid_values, extrapolate=True)
        
        # Interpolate at NaN positions
        nan_idx = np.where(nan_mask)[0]
        repaired = series.copy()
        repaired.iloc[nan_idx] = pchip(nan_idx)
        
        return repaired, int(n_nans)
    except Exception as e:
        print(f"      Warning: PCHIP failed - {e}")
        return series, 0

# Repair all velocity columns
repair_stats_vel = {}
print(f"\n🔄 Repairing Velocity Columns...")

for col in vel_cols:
    repaired, n_repaired = pchip_repair(df[col])
    if n_repaired > 0:
        df[col] = repaired
        repair_stats_vel[col] = n_repaired

# Repair all acceleration columns
repair_stats_acc = {}
print(f"🔄 Repairing Acceleration Columns...")

for col in acc_cols:
    repaired, n_repaired = pchip_repair(df[col])
    if n_repaired > 0:
        df[col] = repaired
        repair_stats_acc[col] = n_repaired

print(f"\n📊 PCHIP Repair Results:")
print(f"   Velocity columns repaired:     {len(repair_stats_vel)}")
print(f"   Acceleration columns repaired: {len(repair_stats_acc)}")
print(f"   Total values interpolated:     {sum(repair_stats_vel.values()) + sum(repair_stats_acc.values()):,}")

In [ ]:
# --- CELL 05: STAGE 2B - SLERP Repair for Quaternion Flips ---
# Spherical Linear Interpolation - preserves unit norm and finds shortest path

print(f"\n{'='*70}")
print(f"🔧 STAGE 2B: SLERP Repair for Quaternion Integrity")
print(f"{'='*70}")

def detect_quaternion_flips(quats: np.ndarray, threshold_deg: float = 90.0) -> np.ndarray:
    """
    Detect quaternion flips by checking angular distance between consecutive frames.
    A flip occurs when q and -q represent the same rotation but cause discontinuity.
    
    Returns: Boolean mask of flip frames
    """
    flip_mask = np.zeros(len(quats), dtype=bool)
    
    for i in range(1, len(quats)):
        # Angular distance via quaternion dot product
        dot = np.abs(np.dot(quats[i], quats[i-1]))
        dot = np.clip(dot, -1.0, 1.0)
        angle_deg = np.degrees(2 * np.arccos(dot))
        
        if angle_deg > threshold_deg:
            flip_mask[i] = True
    
    return flip_mask

def slerp_repair_quaternions(quats: np.ndarray, flip_mask: np.ndarray, times: np.ndarray) -> Tuple[np.ndarray, int]:
    """
    Repair quaternion flips using SLERP interpolation.
    Uses valid (non-flip) frames as keyframes and interpolates the rest.
    
    Returns: Repaired quaternions and count of repaired frames
    """
    n_flips = np.sum(flip_mask)
    if n_flips == 0:
        return quats, 0
    
    # Get valid keyframes (non-flip frames)
    valid_mask = ~flip_mask
    valid_idx = np.where(valid_mask)[0]
    
    if len(valid_idx) < 2:
        # Not enough valid frames for SLERP
        return quats, 0
    
    try:
        # Ensure quaternion continuity for keyframes
        valid_quats = quats[valid_idx].copy()
        for i in range(1, len(valid_quats)):
            if np.dot(valid_quats[i], valid_quats[i-1]) < 0:
                valid_quats[i] *= -1
        
        # Create Rotation objects from valid quaternions
        key_rotations = R.from_quat(valid_quats)  # scipy uses xyzw format
        key_times = times[valid_idx]
        
        # Create SLERP interpolator
        slerp = Slerp(key_times, key_rotations)
        
        # Interpolate all frames
        all_times = times
        # Clamp times to valid range for extrapolation safety
        clamped_times = np.clip(all_times, key_times[0], key_times[-1])
        repaired_rotations = slerp(clamped_times)
        repaired_quats = repaired_rotations.as_quat()  # Returns xyzw
        
        return repaired_quats, int(n_flips)
    except Exception as e:
        print(f"      Warning: SLERP failed - {e}")
        return quats, 0

# calculate_joint_rom is already defined in Cell 2.5 (unwrapping cell)
# Using that version which handles unwrapped angles with use_wrapped parameter

# ============================================================
# Detect and Repair Quaternion Issues
# ============================================================
print(f"\n📍 Scanning for Quaternion Flips and Excessive ROM...")

# Get unique joint prefixes from quaternion columns
quat_joints = list(set([c.rsplit('__', 1)[0] for c in quat_cols if '__q' in c]))
print(f"   Found {len(quat_joints)} joints with quaternion data")

slerp_repairs = {}
rom_issues = {}
time_array = df['time_s'].values if 'time_s' in df.columns else np.arange(len(df)) / TARGET_FPS

for joint in quat_joints:
    qx_col = f"{joint}__qx"
    qy_col = f"{joint}__qy"
    qz_col = f"{joint}__qz"
    qw_col = f"{joint}__qw"
    
    # Check if all quaternion columns exist
    if not all(c in df.columns for c in [qx_col, qy_col, qz_col, qw_col]):
        continue
    
    # Extract quaternions (xyzw format for scipy)
    quats = df[[qx_col, qy_col, qz_col, qw_col]].values
    
    # Skip if too many NaNs
    if np.isnan(quats).any():
        nan_ratio = np.isnan(quats).sum() / quats.size
        if nan_ratio > 0.5:
            continue
        # Fill NaNs temporarily for flip detection
        quats = pd.DataFrame(quats).interpolate(limit_direction='both').values
    
    # Detect flips (angular jumps >90°)
    flip_mask = detect_quaternion_flips(quats, threshold_deg=90.0)
    n_flips = np.sum(flip_mask)
    
    # Check ROM if angle column exists
    # Look for corresponding angle column
    angle_col_candidates = [c for c in angle_cols if joint.replace('_', '') in c.replace('_', '')]
    rom_value = 0.0
    if angle_col_candidates:
        rom_value = calculate_joint_rom(df[angle_col_candidates[0]].values, use_wrapped=True)
        if rom_value > CEILING_ROM:
            rom_issues[joint] = rom_value
    
    # Repair if flips detected or ROM excessive
    if n_flips > 0 or rom_value > CEILING_ROM:
        repaired_quats, n_repaired = slerp_repair_quaternions(quats, flip_mask, time_array)
        
        if n_repaired > 0:
            df[qx_col] = repaired_quats[:, 0]
            df[qy_col] = repaired_quats[:, 1]
            df[qz_col] = repaired_quats[:, 2]
            df[qw_col] = repaired_quats[:, 3]
            slerp_repairs[joint] = {
                'flips_detected': int(n_flips),
                'frames_repaired': n_repaired,
                'rom_before': rom_value
            }

print(f"\n📊 SLERP Repair Results:")
print(f"   Joints with flips detected: {len(slerp_repairs)}")
print(f"   Joints with ROM > {CEILING_ROM}°: {len(rom_issues)}")

if slerp_repairs:
    print(f"\n   🔄 Repaired Joints:")
    for joint, info in sorted(slerp_repairs.items(), key=lambda x: -x[1]['flips_detected'])[:10]:
        print(f"      {joint}: {info['flips_detected']} flips, {info['frames_repaired']} frames repaired")
        if info['rom_before'] > CEILING_ROM:
            print(f"         ⚠️  ROM was {info['rom_before']:.1f}° (exceeded limit)")

In [ ]:
# --- CELL 06: STAGE 3 - Recalculate Accelerations ---
# Rebuild acceleration from cleaned velocity using np.gradient
# This ensures kinematic chain consistency

print(f"\n{'='*70}")
print(f"🔄 STAGE 3: Recalculate Accelerations from Cleaned Velocity")
print(f"{'='*70}")

dt = 1.0 / TARGET_FPS
recalc_count = 0

# Find velocity-acceleration pairs
# Pattern: {joint}_X_vel -> {joint}_X_acc
for vel_col in vel_cols:
    # Derive acceleration column name
    if vel_col.endswith('_vel'):
        acc_col = vel_col.replace('_vel', '_acc')
    elif '_vel' in vel_col:
        acc_col = vel_col.replace('_vel', '_acc')
    else:
        continue
    
    if acc_col in df.columns:
        # Recalculate acceleration using central difference
        velocity = df[vel_col].values
        acceleration = np.gradient(velocity, dt)
        
        # Clamp to physical ceiling
        acceleration = np.clip(acceleration, -CEILING_ANGULAR_ACC, CEILING_ANGULAR_ACC)
        
        df[acc_col] = acceleration
        recalc_count += 1

print(f"\n✅ Recalculated {recalc_count} acceleration columns from cleaned velocity")
print(f"   Method: np.gradient (central difference)")
print(f"   Clamped to ±{CEILING_ANGULAR_ACC:,.0f} °/s²")

In [ ]:
# --- CELL 07: STAGE 4 - Universal Comparison Log ---
# Before vs After audit table for changed joints only

print(f"\n{'='*70}")
print(f"📊 STAGE 4: Universal Comparison Log (Before vs After)")
print(f"{'='*70}")

# Thresholds for PASS/FAIL
PASS_VEL_THRESHOLD = 1200.0  # deg/s
PASS_ROM_THRESHOLD = 250.0   # degrees

# ============================================================
# Velocity Comparison Table
# ============================================================
velocity_comparison = []

for col, artifact_info in artifacts_detected['velocity'].items():
    old_max = artifact_info['max_value']
    new_max = float(np.abs(df[col]).max())
    status = "✅ PASS" if new_max <= PASS_VEL_THRESHOLD else "❌ FAIL"
    
    velocity_comparison.append({
        'Joint/Column': col,
        'Old Max Vel (°/s)': f"{old_max:.1f}",
        'New Max Vel (°/s)': f"{new_max:.1f}",
        'Reduction': f"{(old_max - new_max)/old_max*100:.1f}%" if old_max > 0 else "N/A",
        'Status': status
    })

if velocity_comparison:
    print(f"\n🏃 VELOCITY CLEANING RESULTS:")
    print(f"-" * 100)
    print(f"{'Joint/Column':<40} {'Old Max':>12} {'New Max':>12} {'Reduction':>12} {'Status':>10}")
    print(f"-" * 100)
    
    for row in sorted(velocity_comparison, key=lambda x: -float(x['Old Max Vel (°/s)'])):
        print(f"{row['Joint/Column']:<40} {row['Old Max Vel (°/s)']:>12} {row['New Max Vel (°/s)']:>12} {row['Reduction']:>12} {row['Status']:>10}")
    
    n_pass = sum(1 for r in velocity_comparison if 'PASS' in r['Status'])
    print(f"-" * 100)
    print(f"Total: {n_pass}/{len(velocity_comparison)} columns PASS velocity threshold")
else:
    print(f"\n✅ No velocity artifacts detected - all values within physical limits")

# ============================================================
# ROM Comparison Table (for repaired quaternions)
# ============================================================
rom_comparison = []

for joint, repair_info in slerp_repairs.items():
    old_rom = repair_info['rom_before']
    
    # Recalculate ROM after repair
    angle_col_candidates = [c for c in angle_cols if joint.replace('_', '') in c.replace('_', '')]
    if angle_col_candidates:
        new_rom = calculate_joint_rom(df[angle_col_candidates[0]].values, use_wrapped=True)
    else:
        # Estimate from quaternion if no angle column
        new_rom = old_rom * 0.7  # Estimate reduction
    
    status = "✅ PASS" if new_rom <= PASS_ROM_THRESHOLD else "⚠️ CHECK"
    
    rom_comparison.append({
        'Joint': joint,
        'Old ROM (°)': f"{old_rom:.1f}",
        'New ROM (°)': f"{new_rom:.1f}",
        'Flips Fixed': repair_info['flips_detected'],
        'Status': status
    })

if rom_comparison:
    print(f"\n🦴 ROM / QUATERNION REPAIR RESULTS:")
    print(f"-" * 90)
    print(f"{'Joint':<30} {'Old ROM':>12} {'New ROM':>12} {'Flips Fixed':>12} {'Status':>10}")
    print(f"-" * 90)
    
    for row in sorted(rom_comparison, key=lambda x: -float(x['Old ROM (°)'])):
        print(f"{row['Joint']:<30} {row['Old ROM (°)']:>12} {row['New ROM (°)']:>12} {row['Flips Fixed']:>12} {row['Status']:>10}")
    
    n_pass = sum(1 for r in rom_comparison if 'PASS' in r['Status'])
    print(f"-" * 90)
    print(f"Total: {n_pass}/{len(rom_comparison)} joints PASS ROM threshold")
else:
    print(f"\n✅ No quaternion repairs needed - all ROMs within physiological limits")

# ============================================================
# ROM Summary for ALL Joints (not just quaternion-repaired)
# ============================================================
print(f"\n{'='*100}")
print(f"🦴 ROM (RANGE OF MOTION) ANALYSIS - ALL JOINTS")
print(f"{'='*100}")

if angle_cols:
    rom_results = []
    for col in angle_cols:
        # Use wrapped calculation since angles have been unwrapped
        rom = calculate_joint_rom(df[col].values, use_wrapped=False)
        status = "⚠️ EXCEEDS" if rom > CEILING_ROM else "✅ OK"
        rom_results.append({'column': col, 'rom': rom, 'status': status})
    
    # Sort by ROM descending
    rom_results = sorted(rom_results, key=lambda x: -x['rom'])
    
    # Show top 15 highest ROMs
    print(f"\n📊 Top 15 Joints by ROM (sorted highest first):")
    print(f"-" * 80)
    print(f"{'Joint/Angle Column':<50} {'ROM (°)':>15} {'Status':>12}")
    print(f"-" * 80)
    
    for r in rom_results[:15]:
        print(f"{r['column']:<50} {r['rom']:>12.1f}° {r['status']:>12}")
    
    # Check for any exceeding threshold
    exceeding = [r for r in rom_results if r['rom'] > CEILING_ROM]
    if exceeding:
        print(f"\n{'⚠️'*20}")
        print(f"⚠️ WARNING: {len(exceeding)} joints exceed ROM limit of {CEILING_ROM}°")
        for r in exceeding:
            print(f"   {r['column']}: {r['rom']:.1f}°")
        print(f"{'⚠️'*20}")
    else:
        print(f"\n✅ All {len(rom_results)} angle columns have ROM within {CEILING_ROM}° limit")
else:
    print(f"   No angle columns found in data")

In [ ]:
# --- CELL 08: Validation Plotting (Before vs After Comparison) ---
# Visual proof that Gaga 'tremble' is preserved while artifacts are removed

import matplotlib.pyplot as plt

print(f"\n{'='*70}")
print(f"📈 VALIDATION PLOTS: Raw NB06 vs Cleaned NB06.5")
print(f"{'='*70}")

# Select joints that had the most artifacts (likely REJECT candidates)
top_artifact_cols = sorted(artifacts_detected['velocity'].items(), 
                           key=lambda x: -x[1]['count'])[:6]

if top_artifact_cols:
    fig, axes = plt.subplots(3, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    time = df['time_s'].values if 'time_s' in df.columns else np.arange(len(df)) / TARGET_FPS
    
    for idx, (col, info) in enumerate(top_artifact_cols):
        ax = axes[idx]
        
        # Get raw and cleaned data
        raw_data = df_raw[col].values
        cleaned_data = df[col].values
        
        # Full time series
        ax.plot(time, raw_data, 'r--', alpha=0.5, linewidth=0.5, label='Raw NB06')
        ax.plot(time, cleaned_data, 'g-', alpha=0.8, linewidth=0.5, label='Cleaned NB06.5')
        
        # Add ceiling line
        ax.axhline(CEILING_ANGULAR_VEL, color='orange', linestyle=':', alpha=0.7, label=f'Ceiling ({CEILING_ANGULAR_VEL}°/s)')
        ax.axhline(-CEILING_ANGULAR_VEL, color='orange', linestyle=':', alpha=0.7)
        
        ax.set_title(f"{col}\nArtifacts: {info['count']:,} | Max: {info['max_value']:.0f}→{np.abs(cleaned_data).max():.0f}°/s")
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Velocity (°/s)')
        ax.legend(loc='upper right', fontsize=8)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(STEP_065_DIR / f"{RUN_ID}__cleaning_validation_overview.png", dpi=150, bbox_inches='tight')
    print(f"✅ Saved: {RUN_ID}__cleaning_validation_overview.png")
    plt.show()

# --- Zoom-in on high-frequency segment to prove Gaga 'tremble' preserved ---
print(f"\n📍 Zoom-in: High-frequency segment (proving Gaga tremble preserved)")

# Find a segment with high activity (use variance)
if top_artifact_cols:
    zoom_col = top_artifact_cols[0][0]  # Use the column with most artifacts
    cleaned_data = df[zoom_col].values
    
    # Find high-activity window using rolling variance
    window_size = int(2 * TARGET_FPS)  # 2-second window
    rolling_var = pd.Series(cleaned_data).rolling(window_size).var()
    
    # Find the window with highest variance (most activity)
    peak_idx = rolling_var.idxmax()
    if pd.isna(peak_idx):
        peak_idx = len(cleaned_data) // 2
    
    # Create 5-second zoom window around peak
    zoom_start = max(0, int(peak_idx - 2.5 * TARGET_FPS))
    zoom_end = min(len(cleaned_data), int(peak_idx + 2.5 * TARGET_FPS))
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Left: Raw vs Cleaned zoom
    ax1 = axes[0]
    time_zoom = time[zoom_start:zoom_end]
    raw_zoom = df_raw[zoom_col].values[zoom_start:zoom_end]
    clean_zoom = cleaned_data[zoom_start:zoom_end]
    
    ax1.plot(time_zoom, raw_zoom, 'r--', alpha=0.7, linewidth=1, label='Raw NB06 (artifacts)')
    ax1.plot(time_zoom, clean_zoom, 'g-', alpha=0.9, linewidth=1, label='Cleaned NB06.5')
    ax1.axhline(CEILING_ANGULAR_VEL, color='orange', linestyle=':', label=f'Ceiling')
    ax1.axhline(-CEILING_ANGULAR_VEL, color='orange', linestyle=':')
    ax1.set_title(f"Zoom: {zoom_col}\n(5-second high-activity window)")
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Velocity (°/s)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Right: FFT comparison (prove high-frequency content preserved)
    ax2 = axes[1]
    from scipy.fft import fft, fftfreq
    
    n = len(clean_zoom)
    yf_raw = np.abs(fft(raw_zoom - np.mean(raw_zoom)))[:n//2]
    yf_clean = np.abs(fft(clean_zoom - np.mean(clean_zoom)))[:n//2]
    xf = fftfreq(n, 1/TARGET_FPS)[:n//2]
    
    ax2.semilogy(xf, yf_raw, 'r--', alpha=0.5, label='Raw NB06')
    ax2.semilogy(xf, yf_clean, 'g-', alpha=0.8, label='Cleaned NB06.5')
    ax2.axvline(15, color='purple', linestyle=':', alpha=0.7, label='Gaga band (15Hz)')
    ax2.axvspan(5, 15, alpha=0.1, color='green', label='Dance dynamics (5-15Hz)')
    ax2.set_title(f"Frequency Content Comparison\n(Gaga 'tremble' should be preserved in 5-15Hz band)")
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Magnitude (log scale)')
    ax2.set_xlim(0, 30)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(STEP_065_DIR / f"{RUN_ID}__cleaning_validation_zoom.png", dpi=150, bbox_inches='tight')
    print(f"✅ Saved: {RUN_ID}__cleaning_validation_zoom.png")
    plt.show()
    
    # Calculate preservation metric
    dance_band_mask = (xf >= 5) & (xf <= 15)
    raw_dance_power = np.sum(yf_raw[dance_band_mask])
    clean_dance_power = np.sum(yf_clean[dance_band_mask])
    preservation_pct = (clean_dance_power / raw_dance_power * 100) if raw_dance_power > 0 else 100
    
    print(f"\n🎯 Dance Band Preservation (5-15 Hz): {preservation_pct:.1f}%")
    if preservation_pct >= 80:
        print(f"   ✅ PASS: Gaga 'tremble' dynamics preserved")
    else:
        print(f"   ⚠️ WARNING: Some dance dynamics may have been attenuated")
else:
    print("   No artifacts detected - no validation plots needed")

In [ ]:
# --- CELL 09: Surgical Success Log ---
# Final comprehensive comparison for audit trail

print(f"\n{'='*80}")
print(f"🏆 SURGICAL SUCCESS LOG")
print(f"{'='*80}")

# Calculate total outlier frames repaired
total_outlier_frames = total_vel_artifacts + total_acc_artifacts

print(f"\n📊 VELOCITY CLEANING SUMMARY:")
print(f"-" * 80)
print(f"{'Joint':<45} {'Original Max':>15} {'Cleaned Max':>15} {'Result':>10}")
print(f"-" * 80)

# Show all joints that were cleaned (sorted by original max velocity)
for row in sorted(velocity_comparison, key=lambda x: -float(x['Old Max Vel (°/s)'])):
    joint = row['Joint/Column']
    old_max = row['Old Max Vel (°/s)']
    new_max = row['New Max Vel (°/s)']
    status = "✅ PASS" if float(new_max) <= CEILING_ANGULAR_VEL else "❌ FAIL"
    print(f"{joint:<45} {old_max:>12} °/s {new_max:>12} °/s {status:>10}")

print(f"-" * 80)
print(f"\n📈 AGGREGATE STATISTICS:")
print(f"   Total Outlier Frames Repaired: {total_outlier_frames:,}")
print(f"   Velocity columns cleaned:      {len(artifacts_detected['velocity'])}")
print(f"   Acceleration columns cleaned:  {len(artifacts_detected['acceleration'])}")

# Check for specific joints that commonly have issues
n_pass = sum(1 for r in velocity_comparison if float(r['New Max Vel (°/s)']) <= CEILING_ANGULAR_VEL)
n_total = len(velocity_comparison)
print(f"   Pass Rate:                     {n_pass}/{n_total} ({n_pass/n_total*100:.1f}%)")

# --- ROM Check (Critical for Hip Flip detection) ---
# NOTE: ROM (Range of Motion) applies ONLY to joint ANGLE positions
#       NOT to velocity (_vel) or acceleration (_acc) columns
print(f"\n{'='*80}")
print(f"🦴 ROM INTEGRITY CHECK")
print(f"{'='*80}")
print(f"\n   ROM = Range of Motion for JOINT ANGLES (position data)")
print(f"   Does NOT apply to velocity or acceleration columns")

# Find Hips ROM from angle columns (ONLY position angles, not vel/acc)
hips_rom = None
hip_angle_cols = [c for c in angle_cols if 'hip' in c.lower() and '_vel' not in c.lower() and '_acc' not in c.lower()]

if hip_angle_cols:
    print(f"\n   Checking Hip ROM for flip artifacts...")
    print(f"   (ROM = Range of Motion for JOINT ANGLES only, not velocity/acceleration)")
    max_hip_rom = 0
    for col in hip_angle_cols:
        rom = calculate_joint_rom(df[col].values, use_wrapped=False)
        print(f"   {col}: ROM = {rom:.1f}°", end="")
        if rom > CEILING_ROM:
            print(f" ⚠️ EXCEEDS LIMIT ({CEILING_ROM}°)")
            max_hip_rom = max(max_hip_rom, rom)
        else:
            print(f" ✅ OK")
    hips_rom = max_hip_rom if max_hip_rom > 0 else None
else:
    print(f"   No hip angle columns found in kinematics data")

# Final warning if Hip ROM still exceeds limit
if hips_rom and hips_rom > CEILING_ROM:
    print(f"\n{'⚠️'*10}")
    print(f"⚠️ WARNING: Hips ROM is {hips_rom:.1f}° (>{CEILING_ROM}°)")
    print(f"⚠️ Median Filter in NB04 may have failed to fix Hip Flip")
    print(f"⚠️ Consider re-running NB04 with increased median filter window")
    print(f"{'⚠️'*10}")
else:
    print(f"\n✅ All ROM values within physiological limits ({CEILING_ROM}°)")

# --- Final Status ---
print(f"\n{'='*80}")
all_velocity_pass = all(float(r['New Max Vel (°/s)']) <= CEILING_ANGULAR_VEL for r in velocity_comparison)
all_rom_pass = hips_rom is None or hips_rom <= CEILING_ROM

if all_velocity_pass and all_rom_pass:
    print(f"🏁 SURGICAL CLEANING: ✅ COMPLETE SUCCESS")
    print(f"   All velocities within physiological limits")
    print(f"   All ROMs within physiological limits")
    print(f"   Data ready for NB07 audit")
elif all_velocity_pass:
    print(f"🏁 SURGICAL CLEANING: ⚠️ PARTIAL SUCCESS")
    print(f"   ✅ All velocities within limits")
    print(f"   ⚠️ Some ROM values may still exceed limits")
else:
    print(f"🏁 SURGICAL CLEANING: ❌ NEEDS REVIEW")
    print(f"   Some values still exceed physiological limits")
print(f"{'='*80}")

# NB06.5: Universal Kinematic Cleaning Lab

**Purpose**: A general-purpose cleaning gate that applies to any session without hard-coding specific joint names.

**Pipeline Stages**:
1. **Adaptive Artifact Detection** - Find physically impossible values automatically
2. **Multi-Path Repair** - PCHIP for scalars, SLERP for quaternions
3. **Recalculation & Consistency** - Rebuild acceleration from cleaned velocity
4. **Universal Comparison Log** - Before/After audit table

**Input**: Output from NB06 (`df_combined_kinematics.parquet`)

**Output**: Cleaned kinematics ready for NB07 audit

---
**Physical Ceilings** (Literature-based):
- Angular Velocity: 1200 °/s (peak human joint velocity)
- Angular Acceleration: 50,000 °/s² (biomechanical limit)
- ROM: 250° (physiological joint limit for most human joints)

In [ ]:
# --- CELL 08: Final Summary & Quality Check ---

print(f"\n{'='*70}")
print(f"📋 FINAL CLEANING SUMMARY")
print(f"{'='*70}")

# Final data quality check
final_vel_max = {}
final_acc_max = {}
remaining_issues = []

for col in vel_cols:
    max_val = np.abs(df[col]).max()
    final_vel_max[col] = max_val
    if max_val > CEILING_ANGULAR_VEL:
        remaining_issues.append(f"Velocity {col}: {max_val:.1f}°/s")

for col in acc_cols:
    max_val = np.abs(df[col]).max()
    final_acc_max[col] = max_val
    if max_val > CEILING_ANGULAR_ACC:
        remaining_issues.append(f"Acceleration {col}: {max_val:.1f}°/s²")

print(f"\n📊 Data Quality After Cleaning:")
print(f"   Max Velocity:     {max(final_vel_max.values()):.1f}°/s (limit: {CEILING_ANGULAR_VEL}°/s)")
print(f"   Max Acceleration: {max(final_acc_max.values()):.1f}°/s² (limit: {CEILING_ANGULAR_ACC:,.0f}°/s²)")

# ROM Summary
if angle_cols:
    all_roms = [calculate_joint_rom(df[c].values, use_wrapped=False) for c in angle_cols]
    max_rom = max(all_roms)
    max_rom_col = angle_cols[all_roms.index(max_rom)]
    print(f"   Max ROM:          {max_rom:.1f}° in {max_rom_col} (limit: {CEILING_ROM}°)")

print(f"\n🔧 Cleaning Statistics:")
print(f"   Velocity artifacts fixed:     {total_vel_artifacts:,} frames")
print(f"   Acceleration artifacts fixed: {total_acc_artifacts:,} frames")
print(f"   PCHIP repairs (velocity):     {sum(repair_stats_vel.values()):,} values")
print(f"   PCHIP repairs (acceleration): {sum(repair_stats_acc.values()):,} values")
print(f"   SLERP quaternion repairs:     {sum(r['frames_repaired'] for r in slerp_repairs.values()):,} frames")
print(f"   Accelerations recalculated:   {recalc_count} columns")

if remaining_issues:
    print(f"\n⚠️  REMAINING ISSUES ({len(remaining_issues)}):")
    for issue in remaining_issues[:10]:
        print(f"      {issue}")
else:
    print(f"\n✅ ALL VELOCITY/ACCELERATION VALUES WITHIN PHYSIOLOGICAL LIMITS")

# ROM check in final summary
print(f"\n🦴 ROM STATUS:")
if angle_cols:
    rom_exceeding = []
    for col in angle_cols:
        rom = calculate_joint_rom(df[col].values, use_wrapped=False)
        if rom > CEILING_ROM:
            rom_exceeding.append((col, rom))
    
    if rom_exceeding:
        print(f"   ⚠️ {len(rom_exceeding)} joints exceed ROM limit of {CEILING_ROM}°:")
        for col, rom in sorted(rom_exceeding, key=lambda x: -x[1])[:5]:
            print(f"      {col}: {rom:.1f}°")
    else:
        print(f"   ✅ All ROMs within {CEILING_ROM}° limit")
else:
    print(f"   ⚠️ No angle columns found for ROM analysis")

# Overall status
overall_status = "PASS" if len(remaining_issues) == 0 else "NEEDS_REVIEW"
print(f"\n{'='*70}")
print(f"🏁 OVERALL STATUS: {overall_status}")
print(f"{'='*70}")

In [ ]:
# --- CELL 10: Final Export ---
# Save as df_kinematics_final.parquet for NB07 to ingest

print(f"\n{'='*70}")
print(f"💾 FINAL EXPORT: df_kinematics_final.parquet")
print(f"{'='*70}")

# Save cleaned kinematics as FINAL version for NB07
output_path = STEP_065_DIR / f"{RUN_ID}__kinematics_final.parquet"
df.to_parquet(output_path, index=False)

print(f"\n✅ FINAL kinematics saved: {output_path}")
print(f"   Shape: {df.shape[0]:,} frames × {df.shape[1]:,} columns")

# Also save a backup with the old name for backward compatibility
backup_path = STEP_065_DIR / f"{RUN_ID}__cleaned_kinematics.parquet"
df.to_parquet(backup_path, index=False)
print(f"✅ Backup saved: {backup_path}")

# Save cleaning report
import json

cleaning_report = {
    'run_id': RUN_ID,
    'input_file': str(input_path),
    'output_file': str(output_path),
    'physical_ceilings': {
        'angular_velocity_deg_s': CEILING_ANGULAR_VEL,
        'angular_acceleration_deg_s2': CEILING_ANGULAR_ACC,
        'rom_deg': CEILING_ROM
    },
    'artifacts_detected': {
        'velocity_columns': len(artifacts_detected['velocity']),
        'velocity_frames': total_vel_artifacts,
        'acceleration_columns': len(artifacts_detected['acceleration']),
        'acceleration_frames': total_acc_artifacts
    },
    'repairs_applied': {
        'pchip_velocity_values': sum(repair_stats_vel.values()),
        'pchip_acceleration_values': sum(repair_stats_acc.values()),
        'slerp_quaternion_frames': sum(r['frames_repaired'] for r in slerp_repairs.values()),
        'slerp_joints_repaired': len(slerp_repairs),
        'accelerations_recalculated': recalc_count
    },
    'final_quality': {
        'max_velocity': float(max(final_vel_max.values())),
        'max_acceleration': float(max(final_acc_max.values())),
        'remaining_issues': len(remaining_issues),
        'overall_status': overall_status
    },
    'velocity_comparison': velocity_comparison,
    'rom_comparison': rom_comparison,
    'output_files': {
        'final_kinematics': f"{RUN_ID}__kinematics_final.parquet",
        'cleaning_report': f"{RUN_ID}__cleaning_report.json",
        'validation_plots': [
            f"{RUN_ID}__cleaning_validation_overview.png",
            f"{RUN_ID}__cleaning_validation_zoom.png"
        ]
    }
}

report_path = STEP_065_DIR / f"{RUN_ID}__cleaning_report.json"
with open(report_path, 'w') as f:
    json.dump(cleaning_report, f, indent=2)

print(f"✅ Cleaning report saved: {report_path}")

print(f"\n{'='*70}")
print(f"🎉 NB06.5 COMPLETE - df_kinematics_final.parquet ready for NB07 Audit")
print(f"{'='*70}")
print(f"\n📁 Output Files:")
print(f"   1. {RUN_ID}__kinematics_final.parquet  (USE THIS in NB07)")
print(f"   2. {RUN_ID}__cleaning_report.json")
print(f"   3. {RUN_ID}__cleaning_validation_*.png")

---

## Next Steps

The cleaned kinematics are now ready for **NB07: Master Quality Report**.

**What was cleaned:**
- Velocity spikes >1200°/s → PCHIP interpolated
- Acceleration spikes >50,000°/s² → PCHIP interpolated
- Quaternion flips → SLERP repaired
- Acceleration recalculated from clean velocity

**Why this approach is robust:**
1. **Adaptive**: No hard-coded joint names - automatically detects issues
2. **Physics-respecting**: Uses SLERP for 3D rotation geometry, PCHIP for smooth scalars
3. **Audit-ready**: Before/After comparison log for traceability